# Sequential forward feature selection (SFFS)

In [29]:
!pip install -q mlxtend

In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV, RFE

In [31]:
X, L = load_iris(return_X_y=True)
X_train, X_test, L_train, L_test = train_test_split(X, L, test_size=0.3)

In [32]:
X_train.shape

(105, 4)

In [33]:
knn = KNeighborsClassifier(n_neighbors=1)
sfsl = SFS(knn, k_features=4, forward=True, floating=True, verbose=2, scoring='accuracy', cv=5)
sfsl.fit(X_train, L_train)
results = sfsl.subsets_
print(results)


[2024-03-19 10:01:46] Features: 1/4 -- score: 0.9428571428571428
[2024-03-19 10:01:46] Features: 2/4 -- score: 0.9523809523809523
[2024-03-19 10:01:46] Features: 3/4 -- score: 0.9523809523809523
[2024-03-19 10:01:46] Features: 4/4 -- score: 0.9333333333333332

{1: {'feature_idx': (2,), 'cv_scores': array([0.95238095, 0.95238095, 0.9047619 , 0.95238095, 0.95238095]), 'avg_score': 0.9428571428571428, 'feature_names': ('2',)}, 2: {'feature_idx': (2, 3), 'cv_scores': array([0.95238095, 0.95238095, 0.95238095, 0.95238095, 0.95238095]), 'avg_score': 0.9523809523809523, 'feature_names': ('2', '3')}, 3: {'feature_idx': (1, 2, 3), 'cv_scores': array([0.9047619 , 1.        , 0.95238095, 0.95238095, 0.95238095]), 'avg_score': 0.9523809523809523, 'feature_names': ('1', '2', '3')}, 4: {'feature_idx': (0, 1, 2, 3), 'cv_scores': array([0.9047619 , 0.95238095, 0.95238095, 0.9047619 , 0.95238095]), 'avg_score': 0.9333333333333332, 'feature_names': ('0', '1', '2', '3')}}


# SVM and feature selection with SFFS

## Without feature selection

In [34]:
X, L = load_iris(return_X_y=True)
X_train, X_test, L_train, L_test = train_test_split(X, L, test_size=0.3)

sc = StandardScaler()
sc.fit_transform(X_train)
sc.fit(X_test)

kf = KFold(n_splits=5)
print(kf.get_n_splits(X))

5


In [35]:
def train_svm_model(X, L):
  ACC_WHOLE = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    L_train, L_test = L[train_index], L[test_index]
    svm = SVC(kernel='linear')
    svm.fit(X_train, L_train)
    y_preds = svm.predict(X_test)
    ACC_WHOLE.append(accuracy_score(L_test, y_preds))

  print(f'svm support vectors shape = {svm.support_vectors_.shape}')
  print(f'svm support vectors = {svm.n_support_}')

  return ACC_WHOLE, np.mean(ACC_WHOLE), np.std(ACC_WHOLE)

In [36]:
ACC_WHOLE, mean_acc, std_acc = train_svm_model(X, L)
print(f'Accucracies = {ACC_WHOLE}')
print(f'mean accuracy = {mean_acc}')
print(f'standard deviation = {std_acc}')

svm support vectors shape = (19, 4)
svm support vectors = [3 9 7]
Accucracies = [1.0, 1.0, 0.8666666666666667, 1.0, 0.8666666666666667]
mean accuracy = 0.9466666666666667
standard deviation = 0.06531972647421806


## SFFS

In [37]:
X_train.shape

(105, 4)

In [38]:
svm = SVC(kernel='linear')
sfsl = SFS(svm, k_features=4, forward=True, floating=True, verbose=2, scoring='accuracy', cv=5)
sfsl.fit(X_train, L_train)
results = sfsl.subsets_
print(results)


[2024-03-19 10:01:55] Features: 1/4 -- score: 0.961904761904762
[2024-03-19 10:01:55] Features: 2/4 -- score: 0.961904761904762
[2024-03-19 10:01:55] Features: 3/4 -- score: 0.9523809523809523

{1: {'feature_idx': (3,), 'cv_scores': array([0.95238095, 1.        , 1.        , 0.95238095, 0.9047619 ]), 'avg_score': 0.961904761904762, 'feature_names': ('3',)}, 2: {'feature_idx': (1, 3), 'cv_scores': array([0.95238095, 1.        , 1.        , 0.95238095, 0.9047619 ]), 'avg_score': 0.961904761904762, 'feature_names': ('1', '3')}, 3: {'feature_idx': (0, 2, 3), 'cv_scores': array([1.        , 0.95238095, 0.95238095, 0.95238095, 0.95238095]), 'avg_score': 0.9619047619047618, 'feature_names': ('0', '2', '3')}, 4: {'feature_idx': (0, 1, 2, 3), 'cv_scores': array([1.        , 0.9047619 , 0.95238095, 0.9047619 , 0.95238095]), 'avg_score': 0.9428571428571427, 'feature_names': ('0', '1', '2', '3')}}



[2024-03-19 10:01:55] Features: 3/4 -- score: 0.9619047619047618
[2024-03-19 10:01:56] Features: 4/4 -- score: 0.9428571428571427

In [39]:
# Features: 3/4 -- score: 0.9523809523809523{1: {'feature_idx': (3,), 'cv_scores': array([0.95238095, 1.        , 1.        , 0.95238095, 0.9047619 ]), 'avg_score': 0.961904761904762, 'feature_names': ('3',)}, 2: {'feature_idx': (1, 3), 'cv_scores': array([0.95238095, 1.        , 1.        , 0.95238095, 0.9047619 ]), 'avg_score': 0.961904761904762, 'feature_names': ('1', '3')}, 3: {'feature_idx': (0, 2, 3), 'cv_scores': array([1.        , 0.95238095, 0.95238095, 0.95238095, 0.95238095]), 'avg_score': 0.9619047619047618, 'feature_names': ('0', '2', '3')}, 4: {'feature_idx': (0, 1, 2, 3), 'cv_scores': array([1.        , 0.9047619 , 0.95238095, 0.9047619 , 0.95238095]), 'avg_score': 0.9428571428571427, 'feature_names': ('0', '1', '2', '3')}}
X_train, X_test = X_train[:, [3]], X_test[:, [3]]

In [40]:
X_train.shape

(105, 1)

In [41]:
ACC_WHOLE, mean_acc, std_acc = train_svm_model(X, L)
print(f'Accucracies = {ACC_WHOLE}')
print(f'mean accuracy = {mean_acc}')
print(f'standard deviation = {std_acc}')

svm support vectors shape = (19, 4)
svm support vectors = [3 9 7]
Accucracies = [1.0, 1.0, 0.8666666666666667, 1.0, 0.8666666666666667]
mean accuracy = 0.9466666666666667
standard deviation = 0.06531972647421806


The result of both linear svm without SFFS and with SFFS is the same (mean_accuracy = 0.9466666666666667). However, the SFFS has reduced features.

# How ML algorithms calculate feature importance?

Machine learning models don't directly calculate a single "score" for each feature's importance. Instead, they employ various techniques that assess a feature's contribution to the model's predictions.

1. Feature Coefficients (Linear Models):
In linear models (like linear regression, logistic regression, and SVMs with linear kernels), the coefficients associated with each feature directly reflect their importance. A higher coefficient magnitude indicates a stronger influence on the model's output.

2. Feature Importance Scores (Decision Trees and Random Forests):
Decision trees and random forests measure feature importance by calculating how much a feature split contributes to reducing impurity (e.g., Gini impurity) at each node. The features that lead to the most significant reductions in impurity are considered more important.

3. Permutation Importance (Any Model):
This technique involves shuffling the values of a single feature and observing the change in the model's performance metric (e.g., accuracy, F1-score). A larger decrease in performance indicates that the shuffled feature played a significant role in the model's predictions. This method can be applied to any model type.

4. Feature Selection Techniques:
Algorithms like Sequential Feature Selection (SFS) and Recursive Feature Elimination (RFE) iteratively select or remove features based on their estimated importance. Their output can provide insights into which features contribute the most to model performance.

# Sequential Backward Feature Selection (SBFS)

In [42]:
X, L = load_iris(return_X_y=True)
X_train, X_test, L_train, L_test = train_test_split(X, L, test_size=0.3)

sc = StandardScaler()
sc.fit_transform(X_train)
sc.fit(X_test)

StandardScaler()

In [43]:
svm = SVC(kernel='linear')
sfsl = SFS(svm, k_features=1, forward=False, floating=True, verbose=2, scoring='accuracy', cv=5)
sfsl.fit(X_train, L_train)
results = sfsl.subsets_
print(results)

{4: {'feature_idx': (0, 1, 2, 3), 'cv_scores': array([1.        , 1.        , 0.95238095, 1.        , 1.        ]), 'avg_score': 0.9904761904761905, 'feature_names': ('0', '1', '2', '3')}, 3: {'feature_idx': (0, 2, 3), 'cv_scores': array([1.        , 1.        , 1.        , 1.        , 0.95238095]), 'avg_score': 0.9904761904761905, 'feature_names': ('0', '2', '3')}, 2: {'feature_idx': (2, 3), 'cv_scores': array([1.        , 0.95238095, 1.        , 1.        , 0.95238095]), 'avg_score': 0.980952380952381, 'feature_names': ('2', '3')}, 1: {'feature_idx': (3,), 'cv_scores': array([0.95238095, 0.9047619 , 1.        , 0.95238095, 0.95238095]), 'avg_score': 0.9523809523809523, 'feature_names': ('3',)}}



[2024-03-19 10:03:34] Features: 3/1 -- score: 0.9904761904761905
[2024-03-19 10:03:34] Features: 2/1 -- score: 0.980952380952381
[2024-03-19 10:03:34] Features: 1/1 -- score: 0.9523809523809523

In [44]:
# {4: {'feature_idx': (0, 1, 2, 3), 'cv_scores': array([1.        , 1.        , 0.95238095, 1.        , 1.        ]), 'avg_score': 0.9904761904761905, 'feature_names': ('0', '1', '2', '3')}, 3: {'feature_idx': (0, 2, 3), 'cv_scores': array([1.        , 1.        , 1.        , 1.        , 0.95238095]), 'avg_score': 0.9904761904761905, 'feature_names': ('0', '2', '3')}, 2: {'feature_idx': (2, 3), 'cv_scores': array([1.        , 0.95238095, 1.        , 1.        , 0.95238095]), 'avg_score': 0.980952380952381, 'feature_names': ('2', '3')}, 1: {'feature_idx': (3,), 'cv_scores': array([0.95238095, 0.9047619 , 1.        , 0.95238095, 0.95238095]), 'avg_score': 0.9523809523809523, 'feature_names': ('3',)}}
X_train, X_test = X_train[:, [0, 2, 3]], X_test[:, [0, 2, 3]]

In [45]:
ACC_WHOLE, mean_acc, std_acc = train_svm_model(X, L)
print(f'Accucracies = {ACC_WHOLE}')
print(f'mean accuracy = {mean_acc}')
print(f'standard deviation = {std_acc}')

svm support vectors shape = (19, 4)
svm support vectors = [3 9 7]
Accucracies = [1.0, 1.0, 0.8666666666666667, 1.0, 0.8666666666666667]
mean accuracy = 0.9466666666666667
standard deviation = 0.06531972647421806


The accurcy of SVM with SBFS and SFFS are the same on iris dataset

# Are the results of SFFS and SBFS always the same?
Reasons for Different Results:

1. Search Direction:
SFFS: Starts with an empty set and iteratively adds the most informative feature based on a chosen evaluation criterion.
SBFS: Starts with all features and iteratively removes the least informative feature based on the same criterion.

2. Path Dependence:
The order in which features are added or removed can influence the final selection.
If multiple features have similar importance, the initial choice can determine the subsequent selections.

3. Evaluation Criterion:
Both SFFS and SBFS rely on an evaluation criterion to assess feature importance. Different criteria (e.g., accuracy, F1-score) might prioritize features slightly differently.

# Other methods for feature selection:
1. Filter methods (Correlation, Chi-aquare test, ANOVA, Information gain)
2. Wrapper methods (Forward selection, backward elimination, stepwise selection)
3. Embedded methods (LASSO, Elastic net, Ridge regression)

# Stepwise feature selection
* Stepwise regression is a regression technique used for feature selection, which aims to identify the subset of input features that are most relevant for predicting the output variable.
* stepwise regression systematically selects input features based on their statistical significance and contribution to the model's performance.
1. Forward Selection
2. Backward Elimination
3. Bidirectional Elimination: a combination of forward and backward selection, where the algorithm alternates between adding and removing features until no more changes can be made to improve the model performance.
* To perform stepwise regression, the data must first be prepared by cleaning and preprocessing the data. This includes handling missing values, scaling the data, and encoding categorical variables.
* Common indices used in stepwise regression include the Akaike Information Criterion (AIC), Bayesian Information Criterion (BIC), and modified R-squared.

# Implementing forward stepwise regression
1. Start with an empty set of features.
2. Train the model using one feature at a time, starting with the most statistically significant feature.
3. Evaluate the performance of the model at each step and keep track of the set of features that maximizes the performance.
4. Continue adding features until no more features can be added without reducing the model's performance.

# Implementing backward stepwise regression
1. Start with the full set of features.
2. Train the model using all the features.
3. Evaluate the performance of the model at each step and keep track of the set of features that maximizes the performance.
4. Remove the least statistically significant feature and repeat steps 2 and 3 until no more features can be removed without reducing the model's performance.

# Implementing stepwise regression with both forward and backward selection
1. Start with an empty or full set of features.
2. Perform forward selection until no more features can be added without reducing the model's performance.
3. Perform backward elimination until no more features can be removed without reducing the model's performance.
4. Repeat steps 2 and 3 until no more changes can be made to improve the model performance.

# Choosing the optimal number of features
* To make this decision, we can use a method called "recursive feature elimination" (RFE), which involves repeatedly fitting the model with a decreasing number of features, and selecting the number of features that gives the best performance.

https://dataaspirant.com/stepwise-regression/

In [46]:
X, L = load_iris(return_X_y=True)
X_train, X_test, L_train, L_test = train_test_split(X, L, test_size=0.3)

sc = StandardScaler()
sc.fit_transform(X_train)
sc.fit(X_test)

StandardScaler()

In [47]:
svm = SVC(kernel='linear')
rfe = RFECV(estimator=svm, step=1, cv=5, scoring='accuracy')
rfe.fit(X_train, L_train)
print(rfe.support_)
print(rfe.ranking_)
print(rfe.n_features_)

[ True  True  True  True]
[1 1 1 1]
4


All the 4 features in iris dataset are selected in RFECV method.

In [48]:
ACC_WHOLE, mean_acc, std_acc = train_svm_model(X, L)
print(f'Accucracies = {ACC_WHOLE}')
print(f'mean accuracy = {mean_acc}')
print(f'standard deviation = {std_acc}')

svm support vectors shape = (19, 4)
svm support vectors = [3 9 7]
Accucracies = [1.0, 1.0, 0.8666666666666667, 1.0, 0.8666666666666667]
mean accuracy = 0.9466666666666667
standard deviation = 0.06531972647421806


The RFECV (Recursive Feature Elimination with Cross-Validation) method has the same accuracy as the SBFS and SFFS methods on iris dataset using kfold cross validation and SVM with linear kernel.

# Bi-directional Elimination
It is the combination of both Backward Elimination and Forward Elimination.
1. Select a significance level (SL) to stay and enter in the model. There would be two SLs, one used for backward elimination process and the other one for forward elimination process.
2. Apply the next step of Forward selection. (Select one variable at a time and check for their p-values and select one feature whose p-value is less than SL)
3. Apply all the steps of Backward Elimination. (This step will really start when we’ve selected at least two variables in step(ii) above.) We’ll check if we can get rid of any selected variable with step(ii).
4. Repeat steps(ii) and (iii), until no new variables can be added or no old variable can be exit from the selected features and go to step(v).
5. Stop

https://medium.com/@abhishek.km23/methods-of-feature-selection-3b4c88f0e2d5